# 목차

### 1. Import library

### 2. 네이버 뉴스 경제면 / 금융 섹터 / 기사 수집 함수
- [2.1 Generate date list](#2.1-Generate-date-list)
- [2.2 Generate news title, news url list](#2.2-Generate-news-title,-news-url-list)
- [2.3 Generate news dataframe](#2.3-Generate-news-dataframe)

### 3. 월별 데이터 수집

## 1. Import library

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta

## 2. 네이버 뉴스 경제면 / 금융 섹터 / 기사 수집 함수

### 2.1 Generate date list

In [2]:
# generate date list function

def generate_date_list(startdate, enddate):
    # 시작일과 종료일을 datetime 형식으로 변환
    start = datetime.strptime(startdate, "%Y%m")
    end = datetime.strptime(enddate, "%Y%m")

    # 시작일부터 종료일까지의 날짜 리스트를 생성
    date_list = []
    current = start
    while current <= end:
        # 현재 날짜를 "%Y%m%d" 형식의 문자열로 변환하여 리스트에 추가
        date_list.append(current.strftime("%Y%m%d"))
        
        # 현재 날짜를 하루씩 증가
        current += relativedelta(days=1)
        
        # 다음 날짜의 월이 현재 날짜와 다른 경우, 다음 달의 첫 날로 변경
        if current.month != (current + relativedelta(days=1)).month:
            current = current.replace(day=1) + relativedelta(months=1) # relativedelta : 1달 더함
    
    return date_list

### 2.2 Generate news title, news url list

In [3]:
# generate news title, news url LIST function

def generate_news_list(date_list):
    news_date_li = []
    # 웹 페이지 크롤링 할 때 사용할 페이지 클래스 타입
    page_class_type = ['type06_headline', 'type06']
    
    # 날짜 리스트인 date_list를 반복하면서 뉴스 수집
    for date in tqdm(date_list):
        
        # 날짜별 뉴스 목록을 수집할 URL 생성
        url = f'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&mid=shm&sid1=101&date={date}&page=999'
        
        # HTTP 요청 헤더 설정
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
        
        # HTTP GET 요청을 보내서 웹 페이지의 내용 가져옴
        raw = requests.get(url, headers=headers)
        
        # BeautifulSoup으르 사용해서 웹 페이지의 HTML 파싱
        soup = BeautifulSoup(raw.text, "html.parser")
        
        # 해당 날짜에 해당하는 뉴스 목록의 최대 페이지 수 
        max_page = int(soup.select("#main_content > div.paging > strong")[0].text)
        
        # 각 페이지에서 뉴스 정보 수집
        news_page_li = []
        for page in tqdm(range(1, max_page + 1)):
            for ct in page_class_type:
                for news in range(1, 11): # 한 페이지당 10개의 기사
                    
                    # 페이지별로 뉴스 목록을 가져오는 URL 생성
                    url = f'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&mid=shm&sid1=101&date={date}&page={page}'
                    
                    # HTTP GET 요청을 보내서 해당 페이지의 뉴스 목록을 가져옴
                    raw = requests.get(url, headers=headers)
                    soup = BeautifulSoup(raw.text, "html.parser")
                    
                    # HTML에서 뉴스 제목과 URL 추출
                    news_all = soup.select(f"#main_content > div.list_body.newsflash_body > ul.{ct} > li:nth-child({news}) > dl > dt:nth-child(2) > a")        
                    news_dic = {}
                    
                    # 뉴스 정보가 있는 경우 news_dic 딕셔너리에 key값과 value 매핑
                    if news_all:
                        news_dic['title'] = news_all[0].text.strip()
                        news_dic['url'] = news_all[0]['href']
                        news_dic['pubdate'] = date
                    
                    # 뉴스 정보가 없는 경우 None값을 value로 설정
                    else:
                        news_dic['title'] = None
                        news_dic['url'] = None
                        news_dic['pubdate'] = None
                    
                    # 존재하는 뉴스 정보인 경우 뉴스 리스트에 추가
                    if news_dic['title'] is not None and news_dic['url'] is not None:
                        news_page_li.append(news_dic)
        
        # 해당 날짜의 모든 뉴스 정보를 저장한 리스트를 전체 뉴스 리스트에 추가
        news_date_li.extend(news_page_li)
    
    return news_date_li

### 2.3 Generate news dataframe

In [4]:
# generate news title, url, content, pundate DataFrame Function

def generate_news_content_df(news_date_li):
    news_page_li = []
    
    # 뉴스의 URL을 사용하여 뉴스 페이지에 접근하고 내용 스크랩
    for news in tqdm(news_date_li):
        url = news['url']
        
        # HTTP 요청 헤더 설정
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
        
        # HTTP GET 요청을 보내서 뉴스 페이지의 내용을 가져옴
        raw = requests.get(url, headers=headers)
        soup = BeautifulSoup(raw.text, "html.parser")
        
        # HTML에서 뉴스 내용을 추출
        content = soup.select('#dic_area')
         
        # 뉴스 내용을 딕셔너리에 저장    
        if content:
            news['content'] = content[0].text.strip()
        # 뉴스 내용 없으면 None값 저장
        else:
            news['content'] = None   
            
        # 스크랩한 뉴스 정보를 리스트에 추가
        news_page_li.append(news)
    
    # 뉴스 정보를 데이터프레임으로 변환
    news_df = pd.DataFrame(news_page_li)
    return news_df

## 3. 월별 데이터 수집

### 2021년 6월

In [5]:
# 날짜 리스트 생성
startdate = "202106"
enddate = "202107"
date_list = generate_date_list(startdate, enddate) 
print(date_list)

['20210601', '20210602', '20210603', '20210604', '20210605', '20210606', '20210607', '20210608', '20210609', '20210610', '20210611', '20210612', '20210613', '20210614', '20210615', '20210616', '20210617', '20210618', '20210619', '20210620', '20210621', '20210622', '20210623', '20210624', '20210625', '20210626', '20210627', '20210628', '20210629', '20210701']


In [ ]:
# 뉴스 제목, URL 생성 
news_date_li = generate_news_list(date_list) 

In [7]:
# 뉴스 데이터프레임 생성
news_df = generate_news_content_df(news_date_li) 
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|██████████████████████████████████████████████████████████████████████████████| 9821/9821 [46:22<00:00,  3.53it/s]


,title,url,pubdate,content
0,갈팡질팡 가상화폐…비트코인 4300만원대 이더리움 310만원대 등락,https://n.news.naver.com/mnews/article/009/000...,20210601,[사진 제공 = 연합뉴스] 국내 가상화폐 거래소에서 가상자산 비트코인의 가격이 소폭...
1,은행 앱으로 전기차 가격·대출 조회… 친환경차 금융 상품 봇물,https://n.news.naver.com/mnews/article/005/000...,20210601,보조금·할인정보·사용자 리뷰 등 은행들 손쉬운 모바일 앱 개발 개인간 중고차 직...
2,"""애플카 미공개정보 의혹, 현대차 임원에 직원까지…최소 10명""",https://n.news.naver.com/mnews/article/421/000...,20210601,(서울=뉴스1) 박응진 기자 = '애플카' 관련 미공개정보를 이용한 주식매매로 부당...
3,시중은행 대출 2017년 2월 이후 첫 감소… 청약 자금 유입 일시적 현상,https://n.news.naver.com/mnews/article/022/000...,20210601,서울 시내 시중은행 대출 창구 모습. 뉴스1 지난달 시중은행 가계대출이 201...
4,암호화폐 거래 입출금 64조원… 은행 수수료 `대박`,https://n.news.naver.com/mnews/article/029/000...,20210601,1분기만 작년 총액 2배 육박은행 수수료 수익도 10배\n\n\n\n올해 1분기 국...
...,...,...,...,...
9816,"오늘부터 DSR 강화·LTV 완화… 부족한 대출, 2금융권 몰리나",https://n.news.naver.com/mnews/article/417/000...,20210701,1일부터 차주 단위 총부채원리금상환비율(DSR) 40% 규제가 적용되고 주택담보인정...
9817,오늘부터 가계대출 규제강화…누가 ‘DSR 40% 한도’ 적용 받나,https://n.news.naver.com/mnews/article/028/000...,20210701,은행권 한도 40% 채웠다면 저축은행 등에서 추가대출 가능주부·학생 등은 추정소득으...
9818,수사받는 중에도 벌집계좌 운영…코인 거래소 '천태만상',https://n.news.naver.com/mnews/article/003/001...,20210701,[서울=뉴시스] 금융위원회 내부 모습. (사진=금융위원회 제공) 2021.04.14...
9819,암호화폐 거래소 반격이 시작됐다,https://n.news.naver.com/mnews/article/005/000...,20210701,국내 4대 거래소 합작법인 설립금융 감독당국 감시에 공동 대응군소 거래소는 음성화…...


### 2021년 7월

In [8]:
startdate = "202107"
enddate = "202108"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20210701', '20210702', '20210703', '20210704', '20210705', '20210706', '20210707', '20210708', '20210709', '20210710', '20210711', '20210712', '20210713', '20210714', '20210715', '20210716', '20210717', '20210718', '20210719', '20210720', '20210721', '20210722', '20210723', '20210724', '20210725', '20210726', '20210727', '20210728', '20210729', '20210730', '20210801']


In [ ]:
news_date_li = generate_news_list(date_list)

In [10]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|██████████████████████████████████████████████████████████████████████████████| 9199/9199 [43:39<00:00,  3.51it/s]


,title,url,pubdate,content
0,"신용정보원, 환경부 '플라스틱 줄이기' 캠페인 동참",https://n.news.naver.com/mnews/article/003/001...,20210701,[서울=뉴시스] 최현호 기자 = 한국신용정보원은 환경부가 지난 1월부터 진행 중인 ...
1,"주금공, 성남시와 무주택 청년 주거지원 협약 체결",https://n.news.naver.com/mnews/article/003/001...,20210701,"[서울=뉴시스] 최현호 기자 = 한국주택금융공사(주금공)는 성남시, NH농협은행과 ..."
2,"교보라이프플래닛, 아나필락시스쇼크 진단보험 출시",https://n.news.naver.com/mnews/article/003/001...,20210701,[서울=뉴시스] 신효령 기자 = 교보라이프플래닛생명보험은 아나필락시스쇼크로 인한 진...
3,"이명박 전 대통령 논현동 사저, 111억5600만원에 낙찰",https://n.news.naver.com/mnews/article/421/000...,20210701,지난해 12월 24일 서울 강남구 이 전 대통령 사저의 모습. (뉴스1 DB) 20...
4,2일 대우건설 매각 진행…중흥건설·DS네트웍스 대상,https://n.news.naver.com/mnews/article/018/000...,20210701,[이데일리 문승관 기자] 대우건설 최대주주인 KDB인베스트먼트는 1일 본입찰에 참여...
...,...,...,...,...
9194,"8월도 '방향없는' 박스피에 갇힌다…""가치→성장 주도株 교체기""",https://n.news.naver.com/mnews/article/421/000...,20210801,"""수출·기업실적 호조 vs 코로나19·경기둔화 우려로 등락 반복""IT·커뮤니케이션·..."
9195,테이퍼링 운 띄운 美…'금리 인상→집값 하락' 본격화?,https://n.news.naver.com/mnews/article/003/001...,20210801,"美 연준 ""자산 매입 목표 진전 이뤄""양적 완화 축소 속내 드러냈단 평가""금리 인상..."
9196,[암호화폐 열전]스테이블 코인 '테더',https://n.news.naver.com/mnews/article/003/001...,20210801,[서울=뉴시스]테더 로고(사진=테더 홈페이지) 2021.7.30. photo@new...
9197,"57억원어치 비트코인 기부받은 학교, 받자마자 팔아치운 이유 [임현우의 비트코인 나우]",https://n.news.naver.com/mnews/article/015/000...,20210801,암호화폐 기부받는 비영리단체 확산가격 변동성은 '즉시 현금화'로 해결한국경제신문의 ...


### 2021년 8월

In [19]:
startdate = "202108"
enddate = "202109"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20210801', '20210802', '20210803', '20210804', '20210805', '20210806', '20210807', '20210808', '20210809', '20210810', '20210811', '20210812', '20210813', '20210814', '20210815', '20210816', '20210817', '20210818', '20210819', '20210820', '20210821', '20210822', '20210823', '20210824', '20210825', '20210826', '20210827', '20210828', '20210829', '20210830', '20210901']


In [ ]:
news_date_li = generate_news_list(date_list)

In [21]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|██████████████████████████████████████████████████████████████████████████████| 9308/9308 [44:19<00:00,  3.50it/s]


,title,url,pubdate,content
0,[기하영의 생활 속 카드]'엄카' 대신 중고생 신용카드 어때요,https://n.news.naver.com/mnews/article/277/000...,20210801,"신한·삼성카드, 청소년 가족카드 서비스 선봬신한카드, '마이틴즈' 출시…GS25 혜..."
1,"윤종원 ""中企지원 소명 지킬 것""…기은 60주년 기념사",https://n.news.naver.com/mnews/article/003/001...,20210801,[서울=뉴시스] 박주성 기자 = IBK기업은행은 지난 30일 창립 60주년을 맞아 ...
2,"""카드사·쇼핑몰 장점만 쏙쏙 … 회원수 6배 성장 비결이죠"" [fn이사람]",https://n.news.naver.com/mnews/article/014/000...,20210801,윤치운 삼성카드 이커머스팀 수석직접조달 방식으로 가격 낮추고삼성카드 쇼핑만의 차별성...
3,긴축 전환 초입에는 선진국 주식이 유리,https://n.news.naver.com/mnews/article/028/000...,20210801,Weconomy | 김한진의 자산전략\n\n\n\n미국 연방준비제도(연준·Fed)는...
4,"중흥, 대우건설 인수 MOU 체결…지분율 50.75%",https://n.news.naver.com/mnews/article/003/001...,20210801,상세실사 후 주식매매계약 체결\n\n\n\n[서울=뉴시스] 이영환 기자 = 대우건설...
...,...,...,...,...
9303,"뱅크시 웹사이트 통해 가짜 NFT 3억9천만원에 팔려…""해킹된듯""",https://n.news.naver.com/mnews/article/001/001...,20210901,30대 영국인 이더리움으로 구매\n\n\n\n뱅크시 작품으로 보이는 벽화[로이터=연...
9304,은행들 줄줄이 예·적금 금리 인상,https://n.news.naver.com/mnews/article/022/000...,20210901,신한銀 정기예금 0.25~0.3%P 이어 우리銀도 1일부터 0.1~0.3%P 올려 ...
9305,"대신증권, 서비스 간소화로 비대면 계좌 개설 절반 단축",https://n.news.naver.com/mnews/article/022/000...,20210901,대신증권은 서비스 간소화를 통해 비대면 계좌개설 시간을 절반으로 줄일 수 있게 됐다...
9306,연 7%대 적금 등장… 안전자산으로 ‘머니 무브’ 꿈틀,https://n.news.naver.com/mnews/article/005/000...,20210901,증시 혼란속 안전자산 몰릴 듯\n\n\n\n지난 26일 한국은행이 기준금리 인상을 ...


### 2021년 9월

In [22]:
startdate = "202109"
enddate = "202110"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20210901', '20210902', '20210903', '20210904', '20210905', '20210906', '20210907', '20210908', '20210909', '20210910', '20210911', '20210912', '20210913', '20210914', '20210915', '20210916', '20210917', '20210918', '20210919', '20210920', '20210921', '20210922', '20210923', '20210924', '20210925', '20210926', '20210927', '20210928', '20210929', '20211001']


In [ ]:
news_date_li = generate_news_list(date_list)

In [25]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|██████████████████████████████████████████████████████████████████████████████| 9261/9261 [44:20<00:00,  3.48it/s]


,title,url,pubdate,content
0,"우본, 카카오뱅크 지분 매각 추진…차익만 1조 규모 '잭팟'",https://n.news.naver.com/mnews/article/421/000...,20210901,카뱅 '초기주주'로 참여했던 우본…6년만에 1조 차익\n\n\n\n서울 용산구 카카...
1,"김진욱, 미코바이오메드 주식 보유 지적에 ""처분입장 변함없어""",https://n.news.naver.com/mnews/article/421/000...,20210901,"인사청문과정에서 처분 의사…金 ""일부 처분, 잔여분 매각진행""\n\n\n\n김진욱 ..."
2,8월 주담대 3조8천억↑...“올해 최대 증가폭”,https://n.news.naver.com/mnews/article/119/000...,20210901,가계대출 증가폭 3조5000억원 ‘주춤’\n\n\n\n서울 중구의 한 은행 대출 창...
3,[포토] 신한 창립 20주년 온라인 기념식,https://n.news.naver.com/mnews/article/009/000...,20210901,신한금융그룹이 1일 온라인으로 창립 20주년 기념식을 개최하고 새로운 비전인 '더 ...
4,"신용대출 확 꺾였지만, '미친 집값'에 은행 주담대는 또...",https://n.news.naver.com/mnews/article/008/000...,20210901,"(서울=뉴스1) 박지혜 기자 = 주요 은행의 대출중단에 이어 한도축소, 금리인상 움..."
...,...,...,...,...
9256,‘줄줄 새는 보험금’ 3년 만에 50배 늘어난 갑상선 수술비,https://n.news.naver.com/mnews/article/018/000...,20211001,상반기 실손 합쳐 700억원 넘게 보험금 지급수술 불필요해도 과잉진료하는 곳들 생겨...
9257,뜀뛰는 가계대출금리 2년 만에 최고… 코로나 이전 수준으로,https://n.news.naver.com/mnews/article/005/000...,20211001,"8월 가계대출 평균금리 3.1%주담대 2.88%, 신용대출 3.97%기준금리 인상분..."
9258,은행권 가계대출 평균금리 3.1%… 25개월만에 ‘최고’,https://n.news.naver.com/mnews/article/020/000...,20211001,한달 새 0.12%P 오르며 3%대 진입기준금리 인상-우대금리 축소 영향주담대-신용...
9259,"가벼운 車사고 치료, 과실만큼 보험 부담",https://n.news.naver.com/mnews/article/020/000...,20211001,"정부, ‘나이롱환자’ 유발 자동차보험 개선안 추진2023년 ‘치료비 과실책임주의’ ..."


### 2021년 10월

In [26]:
startdate = "202110"
enddate = "202111"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20211001', '20211002', '20211003', '20211004', '20211005', '20211006', '20211007', '20211008', '20211009', '20211010', '20211011', '20211012', '20211013', '20211014', '20211015', '20211016', '20211017', '20211018', '20211019', '20211020', '20211021', '20211022', '20211023', '20211024', '20211025', '20211026', '20211027', '20211028', '20211029', '20211030', '20211101']


In [ ]:
news_date_li = generate_news_list(date_list)

In [28]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|██████████████████████████████████████████████████████████████████████████████| 9845/9845 [47:17<00:00,  3.47it/s]


,title,url,pubdate,content
0,"교보생명 풋옵션 3차 공판…교보생명 ""어피니티, 가치평가에 관여""",https://n.news.naver.com/mnews/article/277/000...,20211001,4차 공판기일은 10월 15일\n\n\n\n교보생명 기업 가치평가 조작을 둘러싼 세...
1,"교보생명 '풋옵션' 3차공판···""가격 부당개입""VS""약속 안지켜""",https://n.news.naver.com/mnews/article/008/000...,20211001,신창재 교보생명 회장/사진제공=교보생명교보생명 가치평가를 허위로 보고했다는 혐의를 ...
2,“가격 부풀리려 허위보고”… 어피니티에 언성 높인 박진호 교보생명 부사장,https://n.news.naver.com/mnews/article/417/000...,20211001,박진호 교보생명 부사장이 1일 열린 교보생명 기업 가치평가 조작 여부를 둘러싼 3차...
3,"한화생명, 와인, 맞춤운동 등 구독보험 라인업 강화",https://n.news.naver.com/mnews/article/014/000...,20211001,"[파이낸셜뉴스] 한화생명이 와인, 맞춤운동 등 웰니스·헬스케어 취향 맞춤 상품을 출..."
4,"증선위, 회계처리 기준 위반 '코디·나노캠텍' 검찰고발 등 조치",https://n.news.naver.com/mnews/article/003/001...,20211001,[서울=뉴시스] 김제이 기자 = 증권선물위원회는 1일 제18차 정례회의에서 회계처리...
...,...,...,...,...
9840,"류영준 카카오페이 대표 ""이달 코스피 입성… 국민 금융플랫폼으로 키운다""",https://n.news.naver.com/mnews/article/417/000...,20211101,"[CEO포커스] 카카오페이, 오는 3일 코스피 입성\n\n\n\n류영준 카카오페이 ..."
9841,‘쉬운 대출’ 시대 저문다…대출 리모델링 준비해야,https://n.news.naver.com/mnews/article/028/000...,20211101,은행 대출 시 적합성·적절성 원칙 안 지키면 과태료분할상환 확대로 ‘이자만 내는’ ...
9842,‘배당금 준다고?’ 아는 사람만 챙기는 출자금통장,https://n.news.naver.com/mnews/article/018/000...,20211101,출자금통장 A부터 Z까지신협·농협·수협·새마을금고 등 상호금융조합원 가입할 때 자본...
9843,‘대출금리인하요구권’ 손질한다는데… 효과는 글쎄,https://n.news.naver.com/mnews/article/005/000...,20211101,요청건수 느는데 수용률 지속 하락불합리한 거절에도 처벌기준 모호“신용평가 기준 일률...


### 2021년 11월

In [29]:
startdate = "202111"
enddate = "202112"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20211101', '20211102', '20211103', '20211104', '20211105', '20211106', '20211107', '20211108', '20211109', '20211110', '20211111', '20211112', '20211113', '20211114', '20211115', '20211116', '20211117', '20211118', '20211119', '20211120', '20211121', '20211122', '20211123', '20211124', '20211125', '20211126', '20211127', '20211128', '20211129', '20211201']


In [ ]:
news_date_li = generate_news_list(date_list)

In [31]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|████████████████████████████████████████████████████████████████████████████| 10185/10185 [49:17<00:00,  3.44it/s]


,title,url,pubdate,content
0,(영상)은행에 이자 깎아달라 요구하세요…작년 91만명 혜톅,https://n.news.naver.com/mnews/article/018/000...,20211101,신용도 오르면 금융기관에 금리인하 요구 가능홍보부족 탓 유명무실 지적에 금융위 개선...
1,"한 풀 꺾인 신용대출, 높아진 금리에 상환↑",https://n.news.naver.com/mnews/article/018/000...,20211101,"정부 규제, 금리 상승에 따라 상환액이 더 많아져[이데일리 김유성 기자] 지난 8월..."
2,[그래픽] 5대 은행 가계대출 증가 규모,https://n.news.naver.com/mnews/article/001/001...,20211101,(서울=연합뉴스) 장예진 기자 = 1일 KB국민·신한·하나·우리·NH농협 등 5대 ...
3,"“은행의 퇴직연금 반격?”…은행, ETF 투자 길 열리나",https://n.news.naver.com/mnews/article/018/000...,20211101,은행서 증권사로 자금 대이동 우려공격적 투자 MZ세대 이탈 차단하려[이데일리 황병서...
4,"""주택 실수요자 피해 막자"" 은행권 잔금대출 품앗이",https://n.news.naver.com/mnews/article/029/000...,20211101,서울시내 한 은행 대출창구에서 한 고객이 상담을 하고 있다. <연합뉴스> 금융...
...,...,...,...,...
10180,"[단독] 국세청 직원들, 한도 축소 미리 알고 ‘대출 쇼핑’ 의혹은 사실",https://n.news.naver.com/mnews/article/005/000...,20211201,원천징수영수증 발급 건수 분석9월1~9일 총 290건… 일평균 32.2건한도 축소 ...
10181,"마이데이터 시대 개막…내 정보, 한곳에서 한눈에 본다",https://n.news.naver.com/mnews/article/008/000...,20211201,"여러 기관, 기업에 흩어졌던 개인의 정보를 한곳에서 보는 '마이데이터 시대'가 열린..."
10182,"로보어드바이저 업체 파운트 ""투자 상품군에 NFT 포함 검토"" [한경 엣지]",https://n.news.naver.com/mnews/article/015/000...,20211201,AI투자 핀테크 파운트 김영빈 대표 인터뷰올해 투자일임 서비스 새롭게 선보여美 증시...
10183,"[포토] 신한은행, KBO리그 타이틀 스폰서 2년 연장",https://n.news.naver.com/mnews/article/015/000...,20211201,신한은행이 KBO리그 타이틀 스폰서 후원 계약을 2023년까지 2년 더 연장한다. ...


### 2021년 12월

In [32]:
startdate = "202112"
enddate = "202201"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20211201', '20211202', '20211203', '20211204', '20211205', '20211206', '20211207', '20211208', '20211209', '20211210', '20211211', '20211212', '20211213', '20211214', '20211215', '20211216', '20211217', '20211218', '20211219', '20211220', '20211221', '20211222', '20211223', '20211224', '20211225', '20211226', '20211227', '20211228', '20211229', '20211230', '20220101']


In [ ]:
news_date_li = generate_news_list(date_list)

In [34]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|██████████████████████████████████████████████████████████████████████████████| 9866/9866 [47:03<00:00,  3.49it/s]


,title,url,pubdate,content
0,"케이뱅크, 자동 목돈 모으기 '챌린지박스' 출시",https://n.news.naver.com/mnews/article/011/000...,20211201,[서울경제] 케이뱅크는 누구나 쉽고 편하게 돈을 모을 수 있는 자동 목돈 모으기 ‘...
1,"[데일리안 오늘뉴스 종합] 이준석, 잠행 중 부산서 정의화 전 국회의장과 회동, '...",https://n.news.naver.com/mnews/article/119/000...,20211201,국민의힘 이준석 대표 ⓒ데일리안 홍금표 기자[데일리안 = 이건엄 기자] ▲[단독] ...
2,66년생 이재근 행장 추천...KB 세대교체 바람 부나,https://n.news.naver.com/mnews/article/029/000...,20211201,세대교체 신호탄…영업·재무 역량 보유허인 행장 부회장 승진\n\n\n\nKB금융이 ...
3,"(영상)저축銀 예대금리차, 은행의 4배…서민 상대로 이자장사 '논란'",https://n.news.naver.com/mnews/article/018/000...,20211201,저축은행 예대금리차 평균 7.8%p...시중은행의 4배정은보 “저축은행 금리산정체계...
4,사업재편 도모하는 기업…캠코가 자산 인수해 지원,https://n.news.naver.com/mnews/article/277/000...,20211201,한국자산관리공사(캠코)가 신산업 진출을 위해 사업재편을 도모하는 기업의 자산을 인수...
...,...,...,...,...
9861,"[신년사]방문규 수출입은행장 “7000억불 무역강국 도약, 총력 뒷받침”",https://n.news.naver.com/mnews/article/018/000...,20220101,혁신성장 분야에 총 14조 공급중소중견기업에 총 30조 공급‘글로벌 공급망 대응 프...
9862,"[선택 2022-금융①] 대출금리 6%시대...""이자 상환 부담, 실수요자 피해 줄여야""",https://n.news.naver.com/mnews/article/119/000...,20220101,기준금리 인상·대출규제 한파 ‘비상’금리 1%p 뛸 때 이자 13조 늘어나금융지원 ...
9863,"[신년사] 방문규 수출입은행장 ""K-공급망 구축에 총력 지원""",https://n.news.naver.com/mnews/article/001/001...,20220101,방문규 한국수출입은행장 [연합뉴스 자료사진](서울=연합뉴스) 이지헌 기자 = 방문규...
9864,"[신년사]방문규 수은 행장 ""글로벌 공급망 안정화에 총력""",https://n.news.naver.com/mnews/article/003/001...,20220101,[서울=뉴시스] 전신 기자 = 방문규 한국수출입은행장이 지난 10월 13일 서울 여...


### 2022년 1월

In [6]:
startdate = "202201"
enddate = "202202"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20220101', '20220102', '20220103', '20220104', '20220105', '20220106', '20220107', '20220108', '20220109', '20220110', '20220111', '20220112', '20220113', '20220114', '20220115', '20220116', '20220117', '20220118', '20220119', '20220120', '20220121', '20220122', '20220123', '20220124', '20220125', '20220126', '20220127', '20220128', '20220129', '20220130', '20220201']


In [ ]:
news_date_li = generate_news_list(date_list)

In [8]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|██████████████████████████████████████████████████████████████████████████████| 9431/9431 [44:07<00:00,  3.56it/s]


,title,url,pubdate,content
0,"코인 대박, 암호화폐 선택에 달렸다",https://n.news.naver.com/mnews/article/015/000...,20220101,"2022 주식투자 가이드\n\n\n\n사진=REUTERS비트코인 77%, 이더리움 ..."
1,새해 첫 로또 1등 번호 ‘6·11·15·24·32·39’…당첨금 약 14억원(종합),https://n.news.naver.com/mnews/article/018/000...,20220101,[이데일리 배진솔 기자] 새해 첫 날인 1일 ‘제 996회 행복드림 로또’ 추첨 결...
2,996회 로또 1등 ‘6·11·15·24·32·39’…보너스 번호 ‘28’,https://n.news.naver.com/mnews/article/018/000...,20220101,[이데일리 배진솔 기자] 새해 첫날인 1일 ‘제 996회 행복드림 로또’ 추첨 결과...
3,보험료 폭탄 현실로…내년 실손보험 인상률 평균 14.2%,https://n.news.naver.com/mnews/article/277/000...,20220101,1년간 보험료 50%할인으로4세대 전환 유도도\n\n\n\n올해 1~3세대 실손의료...
4,"""MZ 목소리 들어야 산다""…저축銀, '주니어 아이디어 경영' 열풍",https://n.news.naver.com/mnews/article/277/000...,20220101,사진은 JT저축은행 최성욱 대표이사와 주니어 아이디어 공모에 참여해 의견을 제안하고...
...,...,...,...,...
9426,나스닥 3.41% 급등하자 美전기차주 일제히 10%내외 폭등(종합),https://n.news.naver.com/mnews/article/421/000...,20220201,테슬라 주가 일일추이 - 야후 파이낸스 갈무리(서울=뉴스1) 박형기 기자 = 나스닥...
9427,"""경기도의 청년 기본대출, 금리 3% 내외…은행에 이차보전 없어""",https://n.news.naver.com/mnews/article/001/001...,20220201,"소득·자산·신용 무관…경기도, 1월 27일 시중은행 등 대상 설명회이재명 후보의 1..."
9428,"수출입은행 ""올해 조선 신규 수주 25% 감소 전망""",https://n.news.naver.com/mnews/article/001/001...,20220201,"""컨테이너선 수요 감소 여파…중국 장악한 시장 되찾아와야""\n\n\n\n조선업계 수..."
9429,"오미크론 확산에… ""올 설에도 은행 이동점포는 쉽니다""",https://n.news.naver.com/mnews/article/417/000...,20220201,올 추석 연휴에는 대부분의 시중은행들이 이동식 은행점포(이동점포)를 운영하지 않는다...


### 2022년 2월

In [10]:
startdate = "202202"
enddate = "202203"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20220201', '20220202', '20220203', '20220204', '20220205', '20220206', '20220207', '20220208', '20220209', '20220210', '20220211', '20220212', '20220213', '20220214', '20220215', '20220216', '20220217', '20220218', '20220219', '20220220', '20220221', '20220222', '20220223', '20220224', '20220225', '20220226', '20220227', '20220301']


In [ ]:
news_date_li = generate_news_list(date_list)

In [12]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|██████████████████████████████████████████████████████████████████████████████| 7727/7727 [36:40<00:00,  3.51it/s]


,title,url,pubdate,content
0,빚더미 자영업자 어쩌나…금리인상에 이자부담 직격탄,https://n.news.naver.com/mnews/article/009/000...,20220201,10명 중 1명꼴 금리 1%P 오를 때마다 소득 5% 이상 추가 이자비용으로 써야 ...
1,"""아뿔사! 상속 포기 후 발견한 5억 보험금 내 돈이었네""…보험금 수령은",https://n.news.naver.com/mnews/article/009/000...,20220201,"대법원 ""상속인 보험청구권은 고유재산 인정""\n\n\n\n[사진 = 연합뉴스] # ..."
2,청년층 자산형성·창업지원·주택청약에 금융혜택 풍성해진다,https://n.news.naver.com/mnews/article/243/000...,20220201,이자에 저축장려금 추가 지급 적금 출시청년창업기업 투자하는 420억 펀드 조성장기 ...
3,"부산은행, 폐플라스틱 컵 재활용, 제조한 인형 등 기부",https://n.news.naver.com/mnews/article/014/000...,20220201,BNK부산은행은 27일 본점에서 ‘업사이클링 물품 후원’ 전달식을 갖고 ‘리플 캠페...
4,"경남은행, 설프라이즈 외국인 해외송금 사은행사",https://n.news.naver.com/mnews/article/014/000...,20220201,[파이낸셜뉴스] BNK경남은행은 '설프라이즈 외국인 해외송금 사은행사'를 이달 1...
...,...,...,...,...
7722,“소상공인 대출 만기연장·상환유예 더 연장”,https://n.news.naver.com/mnews/article/005/000...,20220301,고승범 금융위원장 밝혀\n\n\n\n고승범(사진) 금융위원장이 28일 소상공인 대출...
7723,[이동훈의 이코노 아웃룩] 러시아 돈줄 끊는 SWIFT 차단… 세계경제도 ‘대혼돈’,https://n.news.naver.com/mnews/article/005/000...,20220301,독일 프랑크푸르트에서 지난 26일(현지시간) 러시아의 우크라이나 침공을 반대하는 집...
7724,미국 ‘12억원 이상 연금 계좌’ 보유한 가입자 26만명,https://n.news.naver.com/mnews/article/005/000...,20220301,연금 선진국 평균 수익률 8% 넘어… 기본 투자상품으로 ‘TDF’ 채택\n\n\n\...
7725,7월 열리는 ‘퇴직연금 투자시대’… 금융시장 신뢰 회복이 과제,https://n.news.naver.com/mnews/article/005/000...,20220301,"[대선 이후 자본시장, 어떻게 바꿔야 하나] <2> ‘연금부자’를 만들자\n\n\n..."


### 2022년 3월

In [13]:
startdate = "202203"
enddate = "202204"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20220301', '20220302', '20220303', '20220304', '20220305', '20220306', '20220307', '20220308', '20220309', '20220310', '20220311', '20220312', '20220313', '20220314', '20220315', '20220316', '20220317', '20220318', '20220319', '20220320', '20220321', '20220322', '20220323', '20220324', '20220325', '20220326', '20220327', '20220328', '20220329', '20220330', '20220401']


In [ ]:
news_date_li = generate_news_list(date_list)

In [15]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|████████████████████████████████████████████████████████████████████████████| 10245/10245 [48:07<00:00,  3.55it/s]


,title,url,pubdate,content
0,게임업계 '별' 김정주 넥슨 창업주 별세 소식에…암호화폐 업계도 '침통',https://n.news.naver.com/mnews/article/421/000...,20220301,김정주 넥슨 창업자 (넥슨 제공) © 뉴스1(서울=뉴스1) 송화연 기자 = 김정주 ...
1,러시아 7개은행 거래 중지·국고채 투자 중단,https://n.news.naver.com/mnews/article/029/000...,20220301,정부가 러시아에 대한 국제사회의 금융 제재에 동참한다는 의사를 표명하고 그에 따른 ...
2,"청년희망적금 외국인 신청 불만… ""연령대나 늘려주지""",https://n.news.naver.com/mnews/article/029/000...,20220301,"183일이상 거소 외국인도 가능0.05% 수준불구 국민청원까지""내가 낸 세금으로 외..."
3,고령화로 20년간 가계소비 연평균 0.9% ↓,https://n.news.naver.com/mnews/article/029/000...,20220301,고령화로 향후 연평균 가계 소비가 줄어들 거란 분석이 나왔다. 연합뉴스 고령화...
4,국내 가상자산 시총 55조… 투자자 80%이상 20~40대,https://n.news.naver.com/mnews/article/029/000...,20220301,"국내 가상자산 시장 규모가 총 55조, 하루 평균 거래규모만도 11조원에 달하는 것..."
...,...,...,...,...
10240,"“4세대 실손 팔면 5만원”… 교보생명 설계사들, 들썩이는 이유",https://n.news.naver.com/mnews/article/417/000...,20220401,교보생명이 4세대 실손으로 전환하는 설계사들에게 건당 5만원의 시책비를 제공한다. ...
10241,“자영업자 부실대출 막아라”… ‘배드뱅크’ 카드 만지작,https://n.news.naver.com/mnews/article/005/000...,20220401,은행 부실 막고 자영업자 재기 효과정부·국책은행 ‘큰짐’… DJ땐 50조공동 설립·...
10242,"美 장단기 국채금리 역전 현상… 시장선 공포, 전문가는 “기우”",https://n.news.naver.com/mnews/article/005/000...,20220401,경기침체까지 상당한 시차 존재\n\n\n\n통상 장기 채권 금리는 단기 금리보다 높...
10243,"케이뱅크, 신용대출 금리 최대 0.4%P 인하… 마통 한도 2억원으로",https://n.news.naver.com/mnews/article/022/000...,20220401,사진=케이뱅크 제공 시중은행에 이어 인터넷전문은행에서도 신용대출 확대에 나섰다...


### 2022년 4월

In [7]:
startdate = "202204"
enddate = "202205"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20220401', '20220402', '20220403', '20220404', '20220405', '20220406', '20220407', '20220408', '20220409', '20220410', '20220411', '20220412', '20220413', '20220414', '20220415', '20220416', '20220417', '20220418', '20220419', '20220420', '20220421', '20220422', '20220423', '20220424', '20220425', '20220426', '20220427', '20220428', '20220429', '20220501']


In [ ]:
news_date_li = generate_news_list(date_list)

In [9]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9859/9859 [46:56<00:00,  3.50it/s]


,title,url,pubdate,content
0,"BNK부산은행, 동백전 서비스 불안정 공식 사과",https://n.news.naver.com/mnews/article/658/000...,20220401,4일 오전 9시되야 서비스 재개될듯서버 4배 증설 및 결제 승인 프로그램 성능 개선...
1,[우크라 침공] 루블화 결제 강행 첫날…러 유럽 가스공급 안끊겨,https://n.news.naver.com/mnews/article/001/001...,20220401,(베를린=연합뉴스) 이율 특파원 = 블라디미르 푸틴 러시아 대통령이 유럽연합(EU)...
2,"운영 차질 빚은 동백전, 4일 오전9시부터 서비스 재개",https://n.news.naver.com/mnews/article/658/000...,20220401,"1일 오후 2시 예정됐던 신규 서비스 정상적으로 작동하지 않아접속 지연, 계좌 등록..."
3,"""금리 통한 가계부채 연착륙"" 일성…추경은 부담 요인",https://n.news.naver.com/mnews/article/422/000...,20220401,[앵커] 청문회 준비를 위해 첫 출근한 이창용 한국은행 총재 후보자가 기준금리 인상...
4,"부산 지역화폐 동백전 운영 차질…""4일 서비스 재개""(종합)",https://n.news.naver.com/mnews/article/001/001...,20220401,"""운영사 변경 첫날 동시 접속 서비스 거래 증가로 오류 발생"" \n\n\n\n부산 ..."
...,...,...,...,...
9854,"코로나發 900조 자영업 대출…새 정부, 연착륙 플랜 시동",https://n.news.naver.com/mnews/article/119/000...,20220501,2금융권 대환·채무 감면 '포인트'은행 등 금융사 출혈 완화 '숙제'\n\n\n\n...
9855,"홍남기 이어 추경호 후보자도 ""MSCI 선진국지수 편입 필요""",https://n.news.naver.com/mnews/article/421/000...,20220501,"청문 서면답변서…""韓경제규모, 자본시장 성숙도 감안 당위성 충분""\n\n\n\n (..."
9856,은행권 첫 '10년만기' 신용대출 나왔다…대출한도 확대 효과,https://n.news.naver.com/mnews/article/001/001...,20220501,"KB, 지난달 29일부터 5년→10년, 신용대출 금리도 다음 주 최대 0.3%p 인..."
9857,[단독]'40년 주담대' 이어 '10년 신용대출' KB국민銀 내놨다,https://n.news.naver.com/mnews/article/008/000...,20220501,[분할상환방식 신용대출 최장 만기 '5→10년' 확대원리금부담 완화로 DSR 규제우...


### 2022년 5월

In [10]:
startdate = "202205"
enddate = "202206"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20220501', '20220502', '20220503', '20220504', '20220505', '20220506', '20220507', '20220508', '20220509', '20220510', '20220511', '20220512', '20220513', '20220514', '20220515', '20220516', '20220517', '20220518', '20220519', '20220520', '20220521', '20220522', '20220523', '20220524', '20220525', '20220526', '20220527', '20220528', '20220529', '20220530', '20220601']


In [ ]:
news_date_li = generate_news_list(date_list)

In [12]:
news_df = generate_news_content_df(news_date_li)
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)
news_df

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10588/10588 [50:51<00:00,  3.47it/s]


,title,url,pubdate,content
0,‘614억 횡령’ 혐의 우리은행 직원 동생도 구속,https://n.news.naver.com/mnews/article/119/000...,20220501,法 “증거인멸 우려”\n\n\n\n회삿돈 614억원을 횡령한 혐의로 구속된 우리은행...
1,`그린 파이낸싱` 잰걸음...한국씨티은행 ESG 행보,https://n.news.naver.com/mnews/article/029/000...,20220501,한국씨티은행이 최근 ESG 행보를 확대하고 있다. 한국씨티은행 제공 한국씨티은...
2,'10년 만기 신용대출' 등장…DSR 규제 피해 한도 확대,https://n.news.naver.com/mnews/article/011/000...,20220501,"[국민銀, 은행권 최초 출시]대출만기 최장 5년서 2배 껑충신한·하나·우리·농협銀 ..."
3,횡령·유용 등 해마다 30~40건… 준법감시·소비자보호 ‘오작동’ [금융사 구멍난 ...,https://n.news.naver.com/mnews/article/014/000...,20220501,(1) 잊을만하면 터지는 금융사고돈 다루는 금융사 사고나면 수백억빼돌린 돈 주식투자...
4,1분기 보험사 연금저축 수익률 선방… 자산운용사는 ‘마이너스’,https://n.news.naver.com/mnews/article/014/000...,20220501,보험사들이 올해 1·4분기 연금저축 수익률이 전년과 비슷한 수준으로 유지하며 선방했...
...,...,...,...,...
10583,"신한금투, 부모 위한 1% 경제교육 유튜브 콘텐츠 공개",https://n.news.naver.com/mnews/article/022/000...,20220601,신한금융투자는 자녀 경제교육에 관심이 있는 부모들을 위해 ‘부모가 꼭 알아야 하는 ...
10584,"신한라이프, 강원 홍천에 ‘빛나는숲 1호’ 조성",https://n.news.naver.com/mnews/article/022/000...,20220601,신한라이프는 강원 홍천 방내리 일대에 1.6㏊ 규모의 ‘빛나는숲 1호’를 조성했다고...
10585,"은행서 명예퇴직한 50대 男, 핀테크 인턴으로 입사한 이유 [긱스]",https://n.news.naver.com/mnews/article/015/000...,20220601,"어니스트펀드 최고령 인턴, 이주성씨2018년 신한銀서 명예퇴직대학 편입 후 빅데이터..."
10586,강원 3월 예금·대출 동반 상승…비은행예금취급기관 증가폭 확대,https://n.news.naver.com/mnews/article/654/000...,20220601,▲ 5만원권[강원도민일보 자료사진]강원지역 3월중 금융기관 여수신이 모두 증가한 가...


### 2022년 6월

In [13]:
startdate = "202206"
enddate = "202207"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20220601', '20220602', '20220603', '20220604', '20220605', '20220606', '20220607', '20220608', '20220609', '20220610', '20220611', '20220612', '20220613', '20220614', '20220615', '20220616', '20220617', '20220618', '20220619', '20220620', '20220621', '20220622', '20220623', '20220624', '20220625', '20220626', '20220627', '20220628', '20220629', '20220701']


In [ ]:
news_date_li = generate_news_list(date_list)

In [13]:
news_df = generate_news_content_df(news_date_li)
news_df

100%|█████████████████████████████████████████████████| 11079/11079 [51:56<00:00,  3.55it/s]


,title,url,pubdate,content
0,"""예금보호한도 올리면 대출이자 올라간다""",https://n.news.naver.com/mnews/article/009/000...,20220601,금융위 국회제출 보고서 입수예금보험료 인상 주장 있지만대출자에 비용 전가 가능성금융...
1,"""코인 실패하지 않으려면 투명성 확보 시급""",https://n.news.naver.com/mnews/article/009/000...,20220601,"오세진 코빗 대표 인터뷰\n\n\n\n ""루나 사태는 투자자들에게 정보 공유가 충분..."
2,[포토] 우리은행 '팬 리포터',https://n.news.naver.com/mnews/article/009/000...,20220601,이원덕 우리은행장(앞줄 왼쪽 넷째)이 지난달 31일 서울 중구 본점에서 열린 '우리...
3,[표] 외국환율고시표 (5월 31일),https://n.news.naver.com/mnews/article/009/000...,20220601,
4,"BNK캐피탈-다나와, 업무협약",https://n.news.naver.com/mnews/article/015/000...,20220601,BNK캐피탈이 가격비교 플랫폼인 다나와와 ‘온라인 자동차금융 플랫폼 활성화를 위한 ...
...,...,...,...,...
11074,"신흥국선 리테일 영업, 선진국선 IB금융 발굴 현지스타일로 최적화… 해외 진출 성과...",https://n.news.naver.com/mnews/article/020/000...,20220701,[The Insight]우리금융그룹은 현지 특성에 맞는 성장 전략을 바탕으로 신흥국...
11075,“자산관리의 시작은 고객”… 부동산·세무·투자상담 종합 서비스,https://n.news.naver.com/mnews/article/020/000...,20220701,[The Insight]\n\n\n\n서울 서초동 ‘TCE시그니처센터’ 내부 모습....
11076,"NH선물, 유쾌한무료급식소에 쌀 지원 나서",https://n.news.naver.com/mnews/article/022/000...,20220701,장승현 NH선물 대표이사(왼쪽)와 안승영 유쾌한공동체 대표(오른쪽)가 29일 경기도...
11077,"경남농협, 농업인 대상 최저 2%대 금리 대출 지원",https://n.news.naver.com/mnews/article/022/000...,20220701,경남농협은 금리상승기를 대비해 농업인의 금융부담완화 및 영농자금 지원을 위한 ‘새출...


In [17]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2022년 7월

In [19]:
startdate = "202207"
enddate = "202208"
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20220701', '20220702', '20220703', '20220704', '20220705', '20220706', '20220707', '20220708', '20220709', '20220710', '20220711', '20220712', '20220713', '20220714', '20220715', '20220716', '20220717', '20220718', '20220719', '20220720', '20220721', '20220722', '20220723', '20220724', '20220725', '20220726', '20220727', '20220728', '20220729', '20220730', '20220801']


In [ ]:
news_date_li = generate_news_list(date_list)

In [21]:
news_df = generate_news_content_df(news_date_li)
news_df

100%|██████████████████████████████████████████████████████████████████████████| 12150/12150 [1:32:09<00:00,  2.20it/s]


,title,url,pubdate,content
0,신한은행도 이상 외화거래 정황... 금감원 검사착수,https://n.news.naver.com/mnews/article/008/000...,20220701,금감원 사옥금융감독원이 우리은행에 이어 신한은행 지점에서도 비정상적인 규모의 외화거...
1,신한은행도 거액 외환 이상 거래 정황…금감원 검사,https://n.news.naver.com/mnews/article/422/000...,20220701,우리은행에 이어 신한은행에서도 통상적인 범위를 벗어난 거액의 외환거래가 지속돼온 정...
2,"윤종원 기업은행장, 베트남 중앙은행 부총재와 중기금융 협력 논의",https://n.news.naver.com/mnews/article/008/000...,20220701,지난 6월 27일 윤종원 기업은행장이 하노이 베트남 중앙은행에서 도안 타이 썬 부총...
3,"금융위, 담보비율 면제 등 증시 변동성 완화 조치 시행",https://n.news.naver.com/mnews/article/003/001...,20220701,기사내용 요약자사주 매수주문 수량 제한 완화도 시행금감원·거래소 합동 공매도 특별점...
4,신한은행에서도 수상한 외환거래 정황…금감원 수시 검사 착수,https://n.news.naver.com/mnews/article/020/000...,20220701,금융감독원 모습. © News1우리은행에 이어 신한은행에서도 일반적인 범위를 벗어난...
...,...,...,...,...
12145,‘특별사료구매자금’ 만기 5년으로 확대…축산농가 ‘숨통’,https://n.news.naver.com/mnews/article/662/000...,20220801,기존 2년만기 상품 대환 가능 3년 거치 2년 균등분할 상환 대출도 연 1% 고정금...
12146,"부산은행 직원, '고객 돈 14억' 애인 계좌로 '꿀꺽'…당국, 지배구조법 개정 검토",https://n.news.naver.com/mnews/article/008/000...,20220801,이복현 금융감독원장이 28일 오전 서울 여의도 국회에서 열린 정무위원회에서 의원 질...
12147,"""'연 6% 신용대출' 이자 못 버틴다"" 빚갚는 영끌족",https://n.news.naver.com/mnews/article/008/000...,20220801,"[올 상반기 신용대출 포함 기타대출 10.2조 감소...7월 DSR 3단계 시행, ..."
12148,[기자수첩] 고금리 시대 저금리 대환대출 중단한 이유는,https://n.news.naver.com/mnews/article/417/000...,20220801,최근 금융권에선 토스뱅크의 대환대출 서비스 중단이 이슈로 떠올랐다. 토스뱅크가 지난...


In [22]:
enddate = int(enddate) - 1 
news_df.to_csv(f'경제면_금융섹터_기사({startdate}-{enddate}).csv',encoding='utf-8-sig')

In [23]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2022년 8월

In [24]:
startdate = '202208'
enddate = '202209'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20220801', '20220802', '20220803', '20220804', '20220805', '20220806', '20220807', '20220808', '20220809', '20220810', '20220811', '20220812', '20220813', '20220814', '20220815', '20220816', '20220817', '20220818', '20220819', '20220820', '20220821', '20220822', '20220823', '20220824', '20220825', '20220826', '20220827', '20220828', '20220829', '20220830', '20220901']


In [ ]:
news_date_li = generate_news_list(date_list)

In [8]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11424/11424 [57:41<00:00,  3.30it/s]


,title,url,pubdate,content
11419,“尹 정부 임기 만료 전 산은 착공 꼭 이룰 것”,https://n.news.naver.com/mnews/article/658/000...,20220901,이성권 부산시 경제부시장\n\n\n\n“산업은행 부산 이전에 대한 대통령과 정부 의...
11420,금융단지 노른자위에 이전…산은 직원 주택 특별분양 지원도,https://n.news.naver.com/mnews/article/658/000...,20220901,부산시 다양한 혜택 마련 준비- 1만여 ㎡에 2028년 사옥 준공- 자녀들 위한 교...
11421,"산은 이전 입법전쟁…부산시는 野와, 노조는 與와도 연대",https://n.news.naver.com/mnews/article/658/000...,20220901,"국회로 넘어간 산은법 운명- 이달 개회 뒤 법 개정 여부 촉각 - 노조, 민주당..."
11422,“배상액 크게 줄여” vs “혈세 수천억 줄 상황”… 당시 관료들 책임론도,https://n.news.naver.com/mnews/article/020/000...,20220901,론스타에 3100억 배상판정 논란“론스타의 주가조작 유죄 인정등… 한국정부 입장 상...
11423,"카카오뱅크, 주담대 금리 최대 0.85%p 내린다",https://n.news.naver.com/mnews/article/022/000...,20220901,카카오뱅크는 주택담보대출 상품 금리를 최대 0.85%포인트 인하한다고 31일 밝혔다...


In [9]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2022년 9월

In [10]:
startdate = '202209'
enddate = '202210'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20220901', '20220902', '20220903', '20220904', '20220905', '20220906', '20220907', '20220908', '20220909', '20220910', '20220911', '20220912', '20220913', '20220914', '20220915', '20220916', '20220917', '20220918', '20220919', '20220920', '20220921', '20220922', '20220923', '20220924', '20220925', '20220926', '20220927', '20220928', '20220929', '20221001']


In [ ]:
news_date_li = generate_news_list(date_list)

In [12]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11979/11979 [55:30<00:00,  3.60it/s]


,title,url,pubdate,content
11974,상호금융 횡령 5년간 286억원인데…고발률은 60%대,https://n.news.naver.com/mnews/article/003/001...,20221001,[서울=뉴시스] 고범준 기자 = 4일 서올의 한 시중은행 외벽에 전세 대출 상품 관...
11975,"두나무, NFT 수익금 산림청 산하기관 기부",https://n.news.naver.com/mnews/article/022/000...,20221001,가상자산 전문기업 두나무는 디지털 아트 전시회 ‘포레스트전’에서 발생한 NFT 낙찰...
11976,"유진투자증권, 유소년 선수·보육원 아동 테니스대회 초청",https://n.news.naver.com/mnews/article/022/000...,20221001,유진투자증권이 ‘유진투자증권 코리아오픈테니스대회 2022’에 테니스 유소년 선수와 ...
11977,"예탁원 ""10월 중 케이카 등 42개사 1.7억주 의무보유 해제""",https://n.news.naver.com/mnews/article/022/000...,20221001,한국예탁결제원은 10월 중 의무보유등록 상장주식 총 1억7296만주(42개사)가 해...
11978,"미래에셋자산운용, 글로벌엑스 재팬 ETF 3종목 동경거래소 신규 상장",https://n.news.naver.com/mnews/article/022/000...,20221001,미래에셋자산운용은 일본 합작법인 ‘Global X Japan(글로벌엑스 재팬)’이 ...


In [13]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2022년 10월

In [5]:
startdate = '202210'
enddate = '202211'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20221001', '20221002', '20221003', '20221004', '20221005', '20221006', '20221007', '20221008', '20221009', '20221010', '20221011', '20221012', '20221013', '20221014', '20221015', '20221016', '20221017', '20221018', '20221019', '20221020', '20221021', '20221022', '20221023', '20221024', '20221025', '20221026', '20221027', '20221028', '20221029', '20221030', '20221101']


In [ ]:
news_date_li = generate_news_list(date_list)

In [7]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11869/11869 [1:06:24<00:00,  2.98it/s]


,title,url,pubdate,content
11864,메리츠증권 3분기 당기순이익 2175억… 전년동기比 13.8%↑,https://n.news.naver.com/mnews/article/022/000...,20221101,메리츠증권이 3분기 연결기준 당기순이익이 2175억원으로 전년동기대비 13.8% 증...
11865,"‘카카오 장애’ 불편 겪은 업비트, 자체 로그인 시스템 도입",https://n.news.naver.com/mnews/article/022/000...,20221101,디지털자산 거래소 업비트가 기존 카카오·애플 ID 로그인을 대체하는 새 로그인 방식...
11866,"미래에셋자산운용, ETF 활용 연금투자 가이드 개정판 발간",https://n.news.naver.com/mnews/article/022/000...,20221101,미래에셋자산운용은 ‘ETF를 활용한 연금투자 가이드’ 개정판을 발간했다고 28일 밝...
11867,‘(무)ABL인터넷네개딱!패키지건강보험(갱신형)’ 출시,https://n.news.naver.com/mnews/article/022/000...,20221101,ABL생명의 온라인채널 보험 브랜드인 ABL인터넷보험은 다양한 보장 급부를 네 가지...
11868,"임기 만료 앞둔 손병환 NH농협금융 회장, 실적 앞세워 연임 청신호?",https://n.news.naver.com/mnews/article/629/000...,20221101,"농협금융, 3분기 누적 당기순이익 1조9717억 원···전년比 8.1%↑\n\n\n..."


In [8]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2022년 11월

In [9]:
startdate = '202211'
enddate = '202212'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20221101', '20221102', '20221103', '20221104', '20221105', '20221106', '20221107', '20221108', '20221109', '20221110', '20221111', '20221112', '20221113', '20221114', '20221115', '20221116', '20221117', '20221118', '20221119', '20221120', '20221121', '20221122', '20221123', '20221124', '20221125', '20221126', '20221127', '20221128', '20221129', '20221201']


In [ ]:
news_date_li = generate_news_list(date_list)

In [11]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12484/12484 [55:29<00:00,  3.75it/s]


,title,url,pubdate,content
12479,"가상화폐거래소 크라켄, 직원 30% 해고…코인 침체·FTX 여파",https://n.news.naver.com/mnews/article/001/001...,20221201,"미국 가상화폐거래소 크라켄 로고[크라켄 페이스북 캡처, 재판매 및 DB 금지](로스..."
12480,"KB증권, 파생상품 양도소득세 신고대행서비스 시행",https://n.news.naver.com/mnews/article/022/000...,20221201,KB증권은 VIP등급 이상 고객을 대상으로 파생상품 양도소득세 신고대행서비스를 시행...
12481,삼성증권 온라인 '비상장주식 Total 솔루션' 서비스 오픈 [투데이 마이머니],https://n.news.naver.com/mnews/article/022/000...,20221201,쏟아지는 새로운 금융상품은 쉽게 지나치기 쉬운 귀찮은 ‘광고’ 취급을 받기 일쑤다....
12482,"BNK경남은행, ‘The Art Card & The Gallery Card’ 출시",https://n.news.naver.com/mnews/article/022/000...,20221201,BNK경남은행이 현직 작가의 작품을 카드 디자인에 반영한 신상품 2종을 내놨다. ...
12483,"옐런 美재무 ""가상화폐 소유자 보호해야…업계 규제 필요""",https://n.news.naver.com/mnews/article/001/001...,20221201,"""실업률 4%는 튼튼한 노동시장…트위터, 콘텐츠 자체 규제해야""""경제계 '中 리스크..."


In [12]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2022년 12월

In [13]:
startdate = '202212'
enddate = '202301'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20221201', '20221202', '20221203', '20221204', '20221205', '20221206', '20221207', '20221208', '20221209', '20221210', '20221211', '20221212', '20221213', '20221214', '20221215', '20221216', '20221217', '20221218', '20221219', '20221220', '20221221', '20221222', '20221223', '20221224', '20221225', '20221226', '20221227', '20221228', '20221229', '20221230', '20230101']


In [ ]:
news_date_li = generate_news_list(date_list)

In [15]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11696/11696 [1:41:57<00:00,  1.91it/s]


,title,url,pubdate,content
11691,"KB금융 윤종규 ""KB,'일상 속 금융플랫폼'으로 한 단계 더 진화""",https://n.news.naver.com/mnews/article/421/000...,20230101,"[신년인터뷰] ""금융정보는 구글, 네이버 아닌 KB플랫폼서 찾게될 것""""통합생보사·..."
11692,"이영종은 끌고, 성대규는 민다.… '출범 2년차' 신한라이프, 대변화 예고",https://n.news.naver.com/mnews/article/417/000...,20230101,신한라이프가 내년 영업력 강화에 본격적으로 나선다. 사진은 신한라이프 을지로 사옥....
11693,"신한금융 조용병 ""신한의 목표는 변하지 않아…비은행 비중 50% 만들 것""",https://n.news.naver.com/mnews/article/421/000...,20230101,"[신년인터뷰] ""2030년까지 비은행 비중 50%, 글로벌 비중 30% 목표""""주요..."
11694,"머스크의 트위터 인수 도운 피델리티, 한달만에 투자액 56% 축소",https://n.news.naver.com/mnews/article/001/001...,20230101,트위터 본사 건물의 회사 로고[로이터 연합뉴스. 재판매 및 DB 금지](로스앤젤레스...
11695,나이롱환자 막고 중복 실손 해지…올해부터 바뀌는 보험제도 알아두세요,https://n.news.naver.com/mnews/article/629/000...,20230101,중복 가입한 회사 실손보험 중지하면 보험료 환급 가능\n\n\n\n올해부터 경상환자...


In [16]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2023년 1월

In [6]:
startdate = '202301'
enddate = '202302'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20230101', '20230102', '20230103', '20230104', '20230105', '20230106', '20230107', '20230108', '20230109', '20230110', '20230111', '20230112', '20230113', '20230114', '20230115', '20230116', '20230117', '20230118', '20230119', '20230120', '20230121', '20230122', '20230123', '20230124', '20230125', '20230126', '20230127', '20230128', '20230129', '20230130', '20230201']


In [ ]:
news_date_li = generate_news_list(date_list)

In [8]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|██████████████████████████████████████████████████████████████████████████| 11130/11130 [1:06:28<00:00,  2.79it/s]


,title,url,pubdate,content
11125,“거래소 파생시장 8시45분 개장 추진”,https://n.news.naver.com/mnews/article/658/000...,20230201,손병두 이사장 기자간담회- “배당 개선·불법공매도 근절- 증권형 토큰 경쟁력 주도 ...
11126,"尹, 금융회장 ‘셀프연임’ 부정적 “지배구조 개선 관심, 관치 아냐”",https://n.news.naver.com/mnews/article/020/000...,20230201,"與일부 “셀프연임, 금융권 모피아 일자리만 만들어”尹, 스튜어드십 거론은 ‘포스코-..."
11127,"하나UBS자산운용, K200액티브 ETF 출시",https://n.news.naver.com/mnews/article/022/000...,20230201,하나UBS자산운용은 KOSPI200지수 대비 초과성과를 추구하는 하나UBS KTOP...
11128,"우리카드, ‘카카오뱅크 우리카드’ 출시…카카오톡 선물하기 50% 할인 제공",https://n.news.naver.com/mnews/article/022/000...,20230201,우리카드가 인터넷전문은행 카카오뱅크와 협업해 ‘카카오뱅크 우리카드’를 출시한다. ...
11129,"어깨 무거운 김문석 SBI저축은행 대표 내정자, 어려운 업황 속 실적 개선 과제",https://n.news.naver.com/mnews/article/629/000...,20230201,"7년 만에 각자대표→단독대표 체제 변화""'안정적 관리'에 집중…리스크 관리 철저히 ..."


In [9]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2023년 2월

In [10]:
startdate = '202302'
enddate = '202303'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20230201', '20230202', '20230203', '20230204', '20230205', '20230206', '20230207', '20230208', '20230209', '20230210', '20230211', '20230212', '20230213', '20230214', '20230215', '20230216', '20230217', '20230218', '20230219', '20230220', '20230221', '20230222', '20230223', '20230224', '20230225', '20230226', '20230227', '20230301']


In [ ]:
news_date_li = generate_news_list(date_list)

In [12]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|████████████████████████████████████████████████████████████████████████████| 10414/10414 [56:29<00:00,  3.07it/s]


,title,url,pubdate,content
10409,신협중앙회 검사·감독이사 이희준,https://n.news.naver.com/mnews/article/005/000...,20230301,신협중앙회는 신임 검사·감독이사로 이희준(사진) 전 금융감독원 부원장보를 선임했다고...
10410,예결원 사장에 ‘낙하산’ 이순호 선임 가결,https://n.news.naver.com/mnews/article/658/000...,20230301,"노조, 출근 저지 투쟁 등 예고\n\n\n\n한국예탁결제원(예결원)은 28일 주주총..."
10411,BNK금융 사외이사 절반 물갈이…빈대인호 맞춤형 재편,https://n.news.naver.com/mnews/article/658/000...,20230301,"이광주·정영석·김병덕 3명 내정- 이사 6명 중 2명, 빈 내정자 인연- 금융권, ..."
10412,HUG 사장 최종 후보자 돌연 사퇴 ‘미스터리’,https://n.news.naver.com/mnews/article/658/000...,20230301,"박동영 전 대우증권 부사장, 내정설·낙하산 등 논란에도 끝끝내 버텨 주총까지 통과-..."
10413,"'애플페이 임박'에 현대카드로 몰려간 MZ세대, 정태영 전략 통할까",https://n.news.naver.com/mnews/article/629/000...,20230301,"업계, 국내 NFC 단말기 보급률 10% 미만MZ세대, 체크카드 회원 수 비해 취급..."


In [13]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2023년 3월

In [14]:
startdate = '202303'
enddate = '202304'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20230301', '20230302', '20230303', '20230304', '20230305', '20230306', '20230307', '20230308', '20230309', '20230310', '20230311', '20230312', '20230313', '20230314', '20230315', '20230316', '20230317', '20230318', '20230319', '20230320', '20230321', '20230322', '20230323', '20230324', '20230325', '20230326', '20230327', '20230328', '20230329', '20230330', '20230401']


In [ ]:
news_date_li = generate_news_list(date_list)

In [16]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|████████████████████████████████████████████████████████████████████████████| 12130/12130 [55:48<00:00,  3.62it/s]


,title,url,pubdate,content
12125,[봄바람 부는 장외주식①]'선학개미' 아시나요,https://n.news.naver.com/mnews/article/003/001...,20230401,기사내용 요약장외주식 거래 활발…저가매수 기간 투자자 유입 \n\n\n\n[서울=뉴...
12126,'320만' 페이코인 투자자 어쩌나… 상폐 소식에 가격 50% 폭락,https://n.news.naver.com/mnews/article/417/000...,20230401,경기도 성남시 분당구 달콤커피 분당서현점을 찾은 한 고객이 페이코인 앱을 이용해 주...
12127,5% 이상 은행 예금 사라졌다,https://n.news.naver.com/mnews/article/003/001...,20230401,기사내용 요약5% 이상 예금 비중 29.7%→ 0.7%로 급감\n\n\n\n[서울=...
12128,"DGB대구은행, 4월 4일부터 신용대출 금리 0.5%p 감면",https://n.news.naver.com/mnews/article/022/000...,20230401,DGB대구은행은 4월 4일부터 가계신용대출 금리를 인하한다고 31일 밝혔다. ...
12129,대부업 철수 속도 내는 OK금융…최윤 회장 꿈 '종합금융그룹' 올라서나,https://n.news.naver.com/mnews/article/629/000...,20230401,3월 예스자산대부 라이선스 반납연내 대부업 조기 철수 목표증권사 인수합병 적극 추진...


In [17]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2023년 4월

In [18]:
startdate = '202304'
enddate = '202305'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20230401', '20230402', '20230403', '20230404', '20230405', '20230406', '20230407', '20230408', '20230409', '20230410', '20230411', '20230412', '20230413', '20230414', '20230415', '20230416', '20230417', '20230418', '20230419', '20230420', '20230421', '20230422', '20230423', '20230424', '20230425', '20230426', '20230427', '20230428', '20230429', '20230501']


In [ ]:
news_date_li = generate_news_list(date_list)

In [20]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|██████████████████████████████████████████████████████████████████████████| 11878/11878 [1:04:17<00:00,  3.08it/s]


,title,url,pubdate,content
11873,‘가상자산’ 법제화 첫 발…투자자 안전망 강화한다,https://n.news.naver.com/mnews/article/658/000...,20230501,"규제 등 포함 법안 국회소위 통과, 손해배상 책임 근거 마련 의미정의에서부터 불공정..."
11874,‘퍼스트리퍼블릭’ 파산수순… 美 은행 올들어 4번째,https://n.news.naver.com/mnews/article/020/000...,20230501,美 14위… 은행위기 재확산 우려\n\n\n\n지난달 26일(현지 시간) 미국 샌프...
11875,소액으로 부동산·명품 등 ‘조각 투자’… MZ세대 몰린다 [마이머니],https://n.news.naver.com/mnews/article/022/000...,20230501,"새로운 투자처로 각광… 분야도 다양화저작권 투자 뮤직카우, 투자자 절반 2030최근..."
11876,"경기창경센터, 경기중기청과 '스타트업 815 IR' 확대 개최",https://n.news.naver.com/mnews/article/030/000...,20230501,지난달 27일 경기창조경제혁신센터가 성남시 판교 창업존에서 기업설명회(IR) 프로그...
11877,[스포츠 꽃피운 금융⑤] 농협 정신 깃든 후원…'라켓 명가' 키웠다,https://n.news.naver.com/mnews/article/629/000...,20230501,후원 선수들 우수한 성적으로 농협금융 브랜드 가치 높여\n\n\n\nNH농협금융은 ...


In [21]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2023년 5월

In [11]:
startdate = '202305'
enddate = '202306'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20230501', '20230502', '20230503', '20230504', '20230505', '20230506', '20230507', '20230508', '20230509', '20230510', '20230511', '20230512', '20230513', '20230514', '20230515', '20230516', '20230517', '20230518', '20230519', '20230520', '20230521', '20230522', '20230523', '20230524', '20230525', '20230526', '20230527', '20230528', '20230529', '20230530', '20230601']


In [ ]:
news_date_li = generate_news_list(date_list)

In [13]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|████████████████████████████████████████████████████████████████████████████| 11292/11292 [55:28<00:00,  3.39it/s]


,title,url,pubdate,content
11287,“美 SVB ‘초고속 파산’… 한국도 은행건전성 감독 강화해야”,https://n.news.naver.com/mnews/article/020/000...,20230601,[2023동아국제금융포럼]노벨경제학상 다이아몬드 기조강연\n\n\n\n31일 서울 ...
11288,챗GPT에 밀린 비트코인…올해 들어 월 기준 첫 하락,https://n.news.naver.com/mnews/article/001/001...,20230601,"지난달 말보다 8% 하락…""AI가 투자자 관심 더 끌어""\n\n\n\n비트코인[로이..."
11289,'위믹스 사기' 코인거래소 3곳 압색…업비트·빗썸·코인원 대상,https://n.news.naver.com/mnews/article/421/000...,20230601,지난해 12월8일 오전 서울 서초구 빗썸 고객지원센터에 위믹스 시세가 표시되고 있다...
11290,"""보험사, 당국 IFRS17 가이드라인으로 이익 눈높이 ↓""",https://n.news.naver.com/mnews/article/018/000...,20230601,"하나증권 ""적용 결과, 2~3분기 결산 이후에 나타날 것""[이데일리 노희준 기자] ..."
11291,"이석준號 농협금융, 역대급 순익 올렸지만 웃지 못하는 이유",https://n.news.naver.com/mnews/article/629/000...,20230601,"건전성 관리 '경고등'…고정이하여신비율 0.39%, 전년 동기比 0.09%포인트↑\..."


In [14]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)

### 2023년 6월

In [15]:
startdate = '202306'
enddate = '202307'
date_list = generate_date_list(startdate, enddate)
print(date_list)

['20230601', '20230602', '20230603', '20230604', '20230605', '20230606', '20230607', '20230608', '20230609', '20230610', '20230611', '20230612', '20230613', '20230614', '20230615', '20230616', '20230617', '20230618', '20230619', '20230620', '20230621', '20230622', '20230623', '20230624', '20230625', '20230626', '20230627', '20230628', '20230629', '20230701']


In [ ]:
news_date_li = generate_news_list(date_list)

In [17]:
news_df = generate_news_content_df(news_date_li)
news_df.tail()

100%|████████████████████████████████████████████████████████████████████████████| 10319/10319 [51:34<00:00,  3.34it/s]


,title,url,pubdate,content
10314,"금융위, 내주 MG손보 ‘부실금융기관 지정’ 법정 공방 마무리…재매각 추진 분수령",https://n.news.naver.com/mnews/article/366/000...,20230701,"서울행정법원, 오는 6일 부실금융기관 지정 소송 판결예금보험공사, 승소 시 7월 내..."
10315,불법사금융으로 내몰리는 취약차주..연동형 법정 최고금리제도가 대안?,https://n.news.naver.com/mnews/article/014/000...,20230701,"지난해 12월 말 기준 대부업체 이용자 수, 98만9000명...지난해 6월 말 대..."
10316,"美SEC, 비트코인 ETF 상장 신청 불허…""명확성·포괄성 부족""",https://n.news.naver.com/mnews/article/421/000...,20230701,"최근 자산운용사들 ETF 상장 신청 이어지는 데 대해 ""부적절""\n\n\n\n비트코..."
10317,"SEC, 비트코인 ETF 상장 불허…""신청서 명확성·설명 부족""",https://n.news.naver.com/mnews/article/001/001...,20230701,비트코인 하락세…SEC 불허 사실 공개 후 한 때 3만 달러 하회\n\n\n\n비트...
10318,"엔화투자 어때…""엔화약세 구조적이나 원엔환율 추가하락 글쎄""",https://n.news.naver.com/mnews/article/018/000...,20230701,기업은행 경제연구소 엔저 현상 분석日 국가부채 많아 긴축시 비용 증가원화 약세도 동...


In [18]:
news_df.to_csv(f'경제면_금융섹터_기사({startdate}).csv',encoding='utf-8-sig', index=False)